<a id='bql-for-equity-top'></a>
# Showcase: Intro to Equities with BQL

This notebook demonstrates several ways you can use BQL to perform equity analyses.

Click **Run all** (<i class="fas fa-forward"></i>) at the top of this window to run all the cells in this notebook.

**Agenda**

* [Setting up your environment](#bql-for-equity-setup)
* [Querying BQL for data](#bql-for-equity-querying)
  * [Querying for an individual equity](#bql-for-equity-querying-bond)
  * [Querying for multiple data items](#bql-for-equity-querying-data)
  * [Querying for multiple tickers](#bql-for-equity-querying-multiple)
  * [Specifying dates](#bql-for-equity-querying-dates)
  * [Retrieving times series data](#bql-for-equity-querying-time-series)
* [Building universes with `univ`](#bql-for-equity-univ)
  * [Building a universe from an existing collection of securities](#bql-for-equity-univ-from-collection)
  * [Determining index membership at a point in time](#bql-for-equity-univ-pit)
  * [Returning composite tickers for a universe with `translatesymbols()`](#bql-for-equity-univ-composite)
  * [Building a universe from peers of an equity ticker](#bql-for-equity-univ-peers)
  * [Importing private portfolios from the Bloomberg Terminal®](#bql-for-equity-univ-portfolio)
* [Filtering, aggregating and ranking data](#bql-for-equity-aggregating)
  * [Performing aggregate analyses with `group()`](#bql-for-equity-aggregating)
  * [Calculating weighted average](#bql-for-equity-aggregating-wavg)
  * [Creating quantiles with `cut()`](#bql-for-equity-aggregating-cut)
  * [Using the `group`-prefix family of functions](#bql-for-equity-aggregating-group-family)
  * [Calculating z-scores](#bql-for-equity-aggregating-z-score)
  * [Using `grouprank()`](#bql-for-equity-aggregating-grouprank)


---
<a id="bql-for-equity-setup" />

## Setting up your environment

Before you can run the examples in this notebook, you must first set up your environment by importing the necessary libraries (`pandas` and `bql`) and connecting to the BQL service:

1. Import pandas if needed for DataFrame handling.
2. Import Bloomberg's PyBQL library, named `bql`.
3. Connect to the BQL Service by creating an instance of the `bql.Service()` class. This allows you to access BQL data items, functions, and universe functions, as well as generate and execute requests. It's recommended to assign the instance to a variable; bq is the common convention.


In [1]:
# Set up your environment
import pandas as pd
import bql

# Connect to the BQL service
bq = bql.Service()

---
<a id="bql-for-equity-querying" />

## Querying BQL for data

After this setting up your environment, you're ready to retrieve BQL data using the following steps:

1. Define your target universe and target data.
2. Create a [`bql.Request`](https://help.bquant.blpprofessional.com/content?id=B2HrxNiSiuKvhYfrCxMfU5) instance with your target universe and target data as arguments.
3. Use the [`bq.execute()`](https://help.bquant.blpprofessional.com/content?id=5CA4pAL4KAoMi3LayCYLqr#bql\.Service\.execute)  function to send the request to the BQL Service.
4. Print the response as a DataFrame to see the output.

There are 3 main types of BQL objects you can use to build a query: 

<table style="width: 100%; border-collapse: collapse; border: solid rgb(0, 0, 0);">
    <tbody>
        <tr>
            <td style="width: 15%; border: 3px solid rgb(0, 0, 0);">
                <div style="text-align: center;"><strong><span style="font-size: 16px;">BQL Object</span></strong></div>
            <td style="width: 15%; border: solid rgb(0, 0, 0);">
                <div style="text-align: center;"><strong><span style="font-size: 16px;">PyBQL Property</span></strong></div>
            <td style="width: 50%; border: solid rgb(0, 0, 0);">
                <div style="text-align: center;"><strong><span style="font-size: 16px;">Action</span></strong></div>
            <td style="width: 20%; border: solid rgb(0, 0, 0);">
                <div style="text-align: center;"><strong><span style="font-size: 16px;">Example</span></strong></div>
        </tr>
        <tr>
            <td style="width: 15%; border: solid rgb(0, 0, 0);">Universe function</td>
            <td style="width: 15%; border: solid rgb(0, 0, 0);"><span style='background-color:gray'>univ</span></td>
            <td style="width: 50%; border: solid rgb(0, 0, 0);">Set criteria to create a target universe of entities.</td>
            <td style="width: 20%; border: solid rgb(0, 0, 0);"><span style='background-color:gray'>bq.univ.members()</span></td>
        </tr>
        <tr>
            <td style="width: 15%; border: solid rgb(0, 0, 0);">Data item</td>
            <td style="width: 15%; border: solid rgb(0, 0, 0);"><span style='background-color:gray'>data</span></td>
            <td style="width: 50%; border: solid rgb(0, 0, 0);">Retrieve a specific data point (e.g., a price) for an entity.</td>
            <td style="width: 20%; border: solid rgb(0, 0, 0);"><span style='background-color:gray'>bq.data.px_last()</span></td>
        </tr>
        <tr>
            <td style="width: 15%; border: solid rgb(0, 0, 0);">Function</td>
            <td style="width: 15%; border: solid rgb(0, 0, 0);"><span style='background-color:gray'>func</span></td>
            <td style="width: 50%; border: solid rgb(0, 0, 0);">Perform a calculation or transformation on a data point.</td>
            <td style="width: 20%; border: solid rgb(0, 0, 0);"><span style='background-color:gray'>bq.func.avg()</span></td>
        </tr>
    </tbody>
</table>



---
<a id="bql-for-equity-querying-equity" />

### Querying for an individual equity 

This example constructs and executes a BQL query to get the name of an individual equity ticker.

<a></a>
<div style="background-color: #353535; white-space: nowrap; padding-top: 0.5rem;">
    <p style="margin-top: -10px; margin-bottom: 0px; margin-left: 5px; padding: 7px; overflow: hidden;">
        <span style="color: white; font-size: 1em; font-weight: bold;"><span style="color: rgb(0, 200, 82);"><strong>&#x24d8;</strong></span>&nbsp;&nbsp;Tip </span>
    </p>
</div>
<div style="background-color: #232323; white-space: nowrap; margin-bottom: 0.5rem;">
    <p style="margin-bottom: 0px; margin-left: 5px; padding: 7px; overflow: hidden;">When using a dictionary to define your target data, the keys become the labels (names) of <br>
        each column (data field), and the values define the target BQL data item.
    </p>
</div>

In [2]:
# Define the target universe, a single equity, using the equity's Bloomberg ticker
universe = 'IBM US Equity'

# Define the target data, the equity's short name, and label the column 'Name'
data_item = {'Name': bq.data.name()}

# Build and execute the BQL query
request = bql.Request(universe, data_item)
response = bq.execute(request)

# Present the data as a Pandas DataFrame
data = response[0].df()
data

,Name
ID,
IBM US Equity,International Business Machine


---
<a id="bql-for-equity-querying-multiple" />

### Other data items and parameters

<table style="width: 100%; border-collapse: collapse; border: solid rgb(0, 0, 0);">
    <tbody>
        <tr>
            <td style="width: 15%; border: 3px solid rgb(0, 0, 0);">
                <div style="text-align: center;"><strong><span style="font-size: 16px;">Data Item</span></strong></div>
            <td style="width: 15%; border: solid rgb(0, 0, 0);">
                <div style="text-align: center;"><strong><span style="font-size: 16px;">Mnemonic</span></strong></div>
            <td style="width: 70%; border: solid rgb(0, 0, 0);">
                <div style="text-align: center;"><strong><span style="font-size: 16px;">Example</span></strong></div>
        </tr>
        <tr>
            <td style="width: 15%; border: solid rgb(0, 0, 0);">Last Price</td>
            <td style="width: 15%; border: solid rgb(0, 0, 0);">PX_LAST</td>
            <td style="width: 70%; border: solid rgb(0, 0, 0);">bq.data.px_last()</td>
        </tr>
        <tr>
            <td style="width: 15%; border: solid rgb(0, 0, 0);">Open Price</td>
            <td style="width: 15%; border: solid rgb(0, 0, 0);">PX_OPEN</td>
            <td style="width: 70%; border: solid rgb(0, 0, 0);">bq.data.px_open()</td>
        </tr>
        <tr>
            <td style="width: 15%; border: solid rgb(0, 0, 0);">High Price</td>
            <td style="width: 15%; border: solid rgb(0, 0, 0);">PX_HIGH</td>
            <td style="width: 70%; border: solid rgb(0, 0, 0);">bq.data.px_high()</td>
        </tr>
        <tr>
            <td style="width: 15%; border: solid rgb(0, 0, 0);">Low Price</td>
            <td style="width: 15%; border: solid rgb(0, 0, 0);">PX_LOW</td>
            <td style="width: 70%; border: solid rgb(0, 0, 0);">bq.data.px_low()</td>
        </tr>
        <tr>
            <td style="width: 15%; border: solid rgb(0, 0, 0);">Volume</td>
            <td style="width: 15%; border: solid rgb(0, 0, 0);">PX_VOLUME</td>
            <td style="width: 70%; border: solid rgb(0, 0, 0);">bq.data.px_volume()</td>
        </tr>
        <tr>
            <td style="width: 15%; border: solid rgb(0, 0, 0);">Traded Value</td>
            <td style="width: 15%; border: solid rgb(0, 0, 0);">TURNOVER</td>
            <td style="width: 70%; border: solid rgb(0, 0, 0);">bq.data.turnover()</td>
        </tr>
        <tr>
            <td style="width: 15%; border: solid rgb(0, 0, 0);">Shares Outstanding</td>
            <td style="width: 15%; border: solid rgb(0, 0, 0);">EQY_SH_OUT</td>
            <td style="width: 70%; border: solid rgb(0, 0, 0);">bq.data.eqy_sh_out()</td>
        </tr>
        <tr>
            <td style="width: 15%; border: solid rgb(0, 0, 0);">Market Capitalisation</td>
            <td style="width: 15%; border: solid rgb(0, 0, 0);">CUR_MKT_CAP</td>
            <td style="width: 70%; border: solid rgb(0, 0, 0);">bq.data.cur_mkt_cap()</td>        
        </tr>
    </tbody>
</table>

<table style="width: 100%; border-collapse: collapse; border: solid rgb(0, 0, 0);">
    <tbody>
        <tr>
            <td style="width: 12.5%; border: 3px solid rgb(0, 0, 0);">
                <div style="text-align: center;"><strong><span style="font-size: 16px;">Parameter Name</span></strong></div>
            <td style="width: 12.5%; border: solid rgb(0, 0, 0);">
                <div style="text-align: center;"><strong><span style="font-size: 16px;">Parameter</span></strong></div>
            <td style="width: 15%; border: solid rgb(0, 0, 0);">
                <div style="text-align: center;"><strong><span style="font-size: 16px;">Default Value</span></strong></div>
            <td style="width: 35%;border: solid rgb(0, 0, 0);">
                <div style="text-align: center;"><strong><span style="font-size: 16px;">Possible Values</span></strong></div>
            <td style="width: 25%;border: solid rgb(0, 0, 0);">
                <div style="text-align: center;"><strong><span style="font-size: 16px;">Example</span></strong></div>
        </tr>
        <tr>
            <td style="width: 12.5%; border: solid rgb(0, 0, 0);">Currency</td>
            <td style="width: 12.5%; border: solid rgb(0, 0, 0);">CURRENCY</td>
            <td style="width: 15%; border: solid rgb(0, 0, 0);">Traded currency of the security</td>
            <td style="width: 35%; border: solid rgb(0, 0, 0);">Currency code you wish for it to be converted to</td>
            <td style="width: 25%; border: solid rgb(0, 0, 0);">bq.data.cur_mkt_cap(CURRENCY=&apos;USD&apos;)</td>
        </tr>
        <tr>
            <td style="width: 12.5%; border: solid rgb(0, 0, 0);">Corporate Action Adjustment</td>
            <td style="width: 12.5%; border: solid rgb(0, 0, 0);">CA_ADJ</td>
            <td style="width: 15%; border: solid rgb(0, 0, 0);">Split</td>
            <td style="width: 35%; border: solid rgb(0, 0, 0);">Raw（Unadjusted）, Full（Adjusted for all corporate actions）, Split（Adjusted for stock splits only）</td>
            <td style="width: 25%; border: solid rgb(0, 0, 0);">CA_ADJ=&apos;Raw&apos;</td>
        </tr>
        <tr>
            <td style="width: 12.5%; border: solid rgb(0, 0, 0);">Date</td>
            <td style="width: 12.5%; border: solid rgb(0, 0, 0);">DATES</td>
            <td style="width: 15%; border: solid rgb(0, 0, 0);">today</td>
            <td style="width: 35%; border: solid rgb(0, 0, 0);">'YYYY-MM-DD' (A specific date in this format) , '-Xd'/'-Xy'（relative date）</td>
            <td style="width: 25%; border: solid rgb(0, 0, 0);">Dates=range('-5d','0d')</td>
        <tr>
            <td style="width: 12.5%; border: solid rgb(0, 0, 0);">Frequency</td>
            <td style="width: 12.5%; border: solid rgb(0, 0, 0);">FRQ</td>
            <td style="width: 15%; border: solid rgb(0, 0, 0);">D</td>
            <td style="width: 35%; border: solid rgb(0, 0, 0);">D, W, M, Q, S, Y</td>
            <td style="width: 25%; border: solid rgb(0, 0, 0);">frq='m'</td>        
        </tr>
    </tbody>
</table>





For more data items, you can run FLDS on the terminal to find available fields compatible in BQuant

---
<a id="bql-for-equity-querying-data" />

### Querying for multiple data items

To query for several data items at once, build your request with a dictionary of the data items. This example constructs a BQL query to return five separate data items about a single equity using its FIGI as the security ticker.


In [3]:
# Define the target universe, a single equity, using the equity's FIGI or ISIN
universe = 'BBG000BLNNH6'

# Define the the target data using a dict to pull multiple data items
data_items = {
    'Name': bq.data.name(),
    'Price': bq.data.px_last(),
    'Volume': bq.data.px_volume(),
    'P/E': bq.data.pe_ratio(),
    'Mkt Cap': bq.data.cur_mkt_cap(),
}

# Build the query and execute the request
request = bql.Request(universe, data_items)
response = bq.execute(request)

Use `pandas` to concatenate all the returned data items into a single dataframe:

In [4]:
data = pd.concat(
    [data_item.df()[data_item.name] for data_item in response], 
    axis=1
)
data

,Name,Price,Volume,P/E,Mkt Cap
ID,,,,,
BBG000BLNNH6,International Business Machine,203.53,3705004.0,21.749219,1.874813e+11


In [5]:
# Calling out the response
response[1]

In [6]:
# Calling out the response for the data item into a data frame, output includes the underlying meta data 
response[1].df()

,DATE,CURRENCY,Price
ID,,,
BBG000BLNNH6,2024-09-10,USD,203.53


In [7]:
# Pulling out only the 'Price' value
response[1].df()['Price']

ID
BBG000BLNNH6    203.53
Name: Price, dtype: float64

In [8]:
# Concatenate the response for Name and Price into a dataframe using pandas
pd.concat([response[0].df()['Name'],response[1].df()['Price']],axis=1)

,Name,Price
ID,,
BBG000BLNNH6,International Business Machine,203.53


In [9]:
# Adding the other data items, however not the most efficient
pd.concat([response[0].df()['Name'],response[1].df()['Price'],response[2].df()['Volume'],response[3].df()['P/E']],axis=1)

,Name,Price,Volume,P/E
ID,,,,
BBG000BLNNH6,International Business Machine,203.53,3705004.0,21.749219


In [10]:
# Using list comprehension to concatenate into a dataframe
pd.concat([data_item.df()[data_item.name] for data_item in response],
          axis=1)

,Name,Price,Volume,P/E,Mkt Cap
ID,,,,,
BBG000BLNNH6,International Business Machine,203.53,3705004.0,21.749219,1.874813e+11


---
<a id="bql-for-equity-querying-multiple" />

### Querying for multiple tickers 

Usually, you'll want to analyze more than one security at a time. You can define a universe of entities by providing BQL with a list of identifiers.


In [11]:
# Define the target universe, multiple securities, using several ID types
universe = [
    'IBM US Equity',    # Bloomberg Ticker
    'US5949181045',     # ISIN
    '037833100',        # CUSIP
    'BBG00DW3SZS1',     # FIGI
    'UKX Index'         # Index ticker
]

# Define your target data
data_items = {
    'Name': bq.data.name(),
    'Mkt Cap': bq.data.cur_mkt_cap(currency='USD'),
    'P/E': bq.data.pe_ratio()
}

# Build and execute the request and display the results
request = bql.Request(universe, data_items)
response = bq.execute(request)

data = pd.concat(
    [data_item.df()[data_item.name] for data_item in response],
    axis=1
)
data

,Name,Mkt Cap,P/E
ID,,,
IBM US Equity,International Business Machine,1.874813e+11,21.749219
US5949181045,Microsoft Corp,3.015732e+12,34.346722
037833100,Apple Inc,3.358746e+12,33.624049
BBG00DW3SZS1,Dell Technologies Inc,7.510821e+10,17.207774
UKX Index,FTSE 100 Index,2.811255e+12,14.594065


---
<a id="bql-for-equity-querying-multiple" />

### Specifying dates

By default, BQL queries return the most up-to-date data available. You can get historical data by specifying either a relative date (e.g., `'-1W'`) or an absolute date (e.g., `'2022-12-07'`) with the [dates](https://help.bquant.blpprofessional.com/bql/parameters/dates-4#parameter-value-type) parameter. The code below returns price of NVIDIA as of 7&nbsp;December&nbsp;2022.

<a></a>
<div style="background-color: #353535; white-space: nowrap; padding-top: 0.5rem;">
    <p style="margin-top: -10px; margin-bottom: 0px; margin-left: 5px; padding: 7px; overflow: hidden;">
        <span style="color: white; font-size: 1em; font-weight: bold;"><span style="color: rgb(0, 200, 82);"><strong>&#x24d8;</strong></span>&nbsp;&nbsp;Tip </span>
    </p>
</div>
<div style="background-color: #232323; white-space: nowrap; margin-bottom: 0.5rem;">
    <p style="margin-bottom: 0px; margin-left: 5px; padding: 7px; overflow: hidden;">To give an absolute date, use the format <code>YYYY-MM-DD</code> in a string.
    </p>
</div>

In [12]:
universe = 'NVDA US Equity'

# Define the date
date = '2022-12-07'

# Prepare the dictionary
data_item = {'Price': bq.data.px_last(dates=date)}

request = bql.Request(universe, data_item)
response = bq.execute(request)

data = response[0].df()
data

,DATE,CURRENCY,Price
ID,,,
NVDA US Equity,2022-12-07,USD,16.12


---
<a id="bql-for-equity-querying-time-series" />

### Retrieving time series data

You can use [func.range()](https://help.bquant.blpprofessional.com/bql/functions/range) to query BQL for time series data over a range of dates. In this example, we pull the closing price of the equity with [data.px_last()](https://help.bquant.blpprofessional.com/bql/data-items/px_last) on each date from 1&nbsp;December to 14&nbsp;December&nbsp;2023. 

BQL allows you to chain multiple functions and data items together. Here we chain [func.dropna()](https://help.bquant.blpprofessional.com/bql/functions/dropna) to remove NA values from the series. In this example, it removes the rows where there are no closing prices, i.e., when the market was closed.

In [33]:
universe = 'NVDA US Equity'

date_range = bq.func.range('2023-12-01', '2023-12-14')
# Define target data including a BQL function to refine the default return
data_item = {'Price': bq.data.px_last(dates=date_range).dropna()}

request  = bql.Request(universe, data_item)
response = bq.execute(request)

data = response[0].df()

# Display first 5 rows of the resulting dataframe
data.head()

,DATE,CURRENCY,Price
ID,,,
NVDA US Equity,2023-12-01,USD,46.765
NVDA US Equity,2023-12-04,USD,45.510
NVDA US Equity,2023-12-05,USD,46.566
NVDA US Equity,2023-12-06,USD,45.503
NVDA US Equity,2023-12-07,USD,46.596


You can also specify date ranges using relative dates. This example pulls an equity's closing price for the past week then chains [func.pct_chg()](https://help.bquant.blpprofessional.com/bql/functions/pct_chg) to calculate the percentage change in its closing price over that time.

In this example, `px_last()` uses the `fill='prev'` arguement to replace missing data points with the last available value.

In [14]:
universe = 'NVDA US Equity'

last_week = bq.func.range('-1w', '0d')
# Define target data including a BQL function that aggregates the data into a 
# single row containing the percent change between the first and last values
data_item = {
    'Price Change': bq.data.px_last(dates=last_week, fill='prev').pct_chg()
}

request = bql.Request(universe, data_item)
response = bq.execute(request)

data = response[0].df()
data

,DATE,CURRENCY,Price Change
ID,,,
NVDA US Equity,2024-09-10,None,-1.416667


---
<a id="bql-for-equity-querying-fundamental" />

### Retrieving fundamental data

Fundamental and estimate data items accept specific parameters to control the period and type of returned data. The most common are:

- [fa_period_type](https://help.bquant.blpprofessional.com/bql/parameters/fa_period_type): The specific filing type required
- [fa_period_offset](https://help.bquant.blpprofessional.com/bql/parameters/fa_period_offset): The number of fundamental periods offset from a set anchor point
- [fa_period_reference](https://help.bquant.blpprofessional.com/bql/parameters/fa_period_reference): The specific fundamental periods required

Use [data.sales_rev_turn()](https://help.bquant.blpprofessional.com/bql/data-items/sales_rev_turn) to return a time series of the quarterly sales figures for Apple from 2021&nbsp;Q1 to 2023&nbsp;Q4.

In [15]:
universe = 'AAPL US Equity'

# Quarterly sales figures from Q1 2021 to Q4 2023
sales = bq.data.sales_rev_turn(
    fa_period_reference=bq.func.range('2021Q1', '2023Q4'),
    fa_period_type='Q'
)
data_item = {'Sales': sales}

request = bql.Request(universe, data_item)
response = bq.execute(request)

data = response[0].df()
data.head()

,REVISION_DATE,AS_OF_DATE,PERIOD_END_DATE,CURRENCY,Sales
ID,,,,,
AAPL US Equity,2021-01-28,2024-09-10,2020-12-25,USD,1.114390e+11
AAPL US Equity,2022-04-29,2024-09-10,2021-03-27,USD,8.958400e+10
AAPL US Equity,2022-07-29,2024-09-10,2021-06-26,USD,8.143400e+10
AAPL US Equity,2022-10-28,2024-09-10,2021-09-25,USD,8.336000e+10
AAPL US Equity,2023-02-03,2024-09-10,2021-12-25,USD,1.239450e+11


---
<a id="bql-for-equity-querying-estimate" />

### Retrieving estimate data

By default, BQL returns estimates when provided a fundamental data item and a period in the future. 

This code example uses [data.ebitda()](https://help.bquant.blpprofessional.com/bql/data-items/ebitda) to return a series of EBITDA estimates for Apple for the next three years.

In [16]:
universe = 'AAPL US Equity'

# EBITDA estimate for next 3 annual periods
ebitda_estimate = bq.data.ebitda(
    fa_period_offset=bq.func.range(1, 3),
    fa_period_type='A'
)
data_item = {'EBITDA Est': ebitda_estimate}

request = bql.Request(universe, data_item)
response = bq.execute(request)

data = response[0].df()
data.head()

,REVISION_DATE,AS_OF_DATE,PERIOD_END_DATE,CURRENCY,EBITDA Est
ID,,,,,
AAPL US Equity,2024-09-09,2024-09-10,2024-09-30,USD,1.341733e+11
AAPL US Equity,2024-09-09,2024-09-10,2025-09-30,USD,1.478841e+11
AAPL US Equity,2024-09-09,2024-09-10,2026-09-30,USD,1.547757e+11


---
<a id="bql-for-equity-univ"></a>

## Building universes with `univ`

Rather than specifying lists of individual equities, you can quickly build universes of relevant instruments with BQL's [univ](https://help.bquant.blpprofessional.com/content?id=XcGc6XrcLwzzUDzXyxwFcz) accessor property. When querying large universes, use the [univ.filter()](https://help.bquant.blpprofessional.com/bql/universe_functions/filter) function to limit your results to only relevant securities.

- Use [univ.members()](https://help.bquant.blpprofessional.com/bql/universe_functions/members) to pull all members of an index.

- Use [univ.peers()](https://help.bquant.blpprofessional.com/bql/universe_functions/peers) to pull equities that belong to the peer group of a chosen equity. You can specify either Bloomberg-defined peers or custom-defined peers.

- Use [univ.equitiesuniv()](https://help.bquant.blpprofessional.com/bql/universe_functions/equitiesuniv) to pull tickers from the entire equities universe. (You'll typically want to call this method with 'active' and 'primary' as parameters to only include actively traded and the primary listed tickers.)

---
<a id="bql-for-equity-univ-from-collection" />

### Building a universe from an existing collection of securities

Use [univ.members()](https://help.bquant.blpprofessional.com/bql/universe_functions/members) to decompose an existing collection of securities (e.g., an index or portfolio) into its constituent members. The following example uses `fa_period_type='BA'` to return the latest calenderized annual sales figures for the members of an index.

In [17]:
universe = bq.univ.members('INDU Index')
data_item = {'Sales': bq.data.sales_rev_turn(fa_period_type='BA')}

request = bql.Request(universe, data_item)
response = bq.execute(request)

data = response[0].df()
data.sort_values('ID').head()

,REVISION_DATE,AS_OF_DATE,PERIOD_END_DATE,CURRENCY,Sales
ID,,,,,
AAPL UQ Equity,2024-05-02,2024-09-10,2023-12-31,USD,3.867033e+11
AMGN UQ Equity,2024-02-14,2024-09-10,2023-12-31,USD,2.819000e+10
AMZN UQ Equity,2024-02-01,2024-09-10,2023-12-31,USD,5.747850e+11
AXP UN Equity,2024-02-09,2024-09-10,2023-12-31,USD,6.736400e+10
BA UN Equity,2024-01-31,2024-09-10,2023-12-31,USD,7.779400e+10


---
<a id="bql-for-equity-univ-pit" />

### Determining index membership at a point in time

Index membership changes over time, and you can use the `dates` parameter with [univ.members()](https://help.bquant.blpprofessional.com/bql/universe_functions/members) to retrieve the members of an index on a certain date.

This example gets the members of an index as of 1&nbsp;October&nbsp;2023 and queries BQL for most recent annual EPS and current annual estimated EPS as of that date.

In [18]:
universe = bq.univ.members('INDU Index', dates='2023-10-01')
data_items = {
    'Annual EPS': bq.data.is_eps(fa_period_type='A',
                                 dates='2023-10-01'),
    'Estimated EPS': bq.data.is_eps(fa_period_offset='1',
                                    fa_period_type='A',
                                    dates='2023-10-01')
}
    
request = bql.Request(universe, data_items)
response = bq.execute(request)    

data = pd.concat(
    [data_item.df()[data_item.name] for data_item in response],
    axis=1
)
data.sort_values('ID').head()

,Annual EPS,Estimated EPS
ID,,
AAPL UQ Equity,6.15,6.069524
AMGN UQ Equity,12.18,14.850000
AXP UN Equity,9.86,11.125417
BA UN Equity,-8.30,-1.298667
CAT UN Equity,12.72,18.721000


---
<a id="bql-for-equity-univ-composite" />

### Returning composite tickers for a universe with `translatesymbols()`

By default, `members()` will return the specific exchange listing of the ticker. Use [univ.translatesymbols()](https://help.bquant.blpprofessional.com/bql/universe_functions/translatesymbols) to transform a ticker from one format to another.

Use the `targetidtype` parameter to specify the desired ticker:

- Passing `'COMPOSITE'` returns the composite exchange ticker.
- Passing `'FUNDAMENTALTICKER'` returns the main ticker storing fundamental data.

This example gets the members of the Dow Jones Industrial Average index, transforms the tickers to their composite exchange equivalents, and queries BQL for the volume on 17&nbsp;June&nbsp;2024. Note how the resulting tickers are from the 'US' composite exchange.

In [19]:
universe = (
    bq.univ.members('INDU Index')
    .translatesymbols(targetidtype='COMPOSITE')
)
data_item = {'Volume': bq.data.px_volume(dates='2024-06-17')}
    
request = bql.Request(universe, data_item)
response = bq.execute(request)

data = pd.concat(
    [data_item.df()[data_item.name] for data_item in response],
    axis=1
)
data.sort_values('ID').head()

,Volume
ID,
AAPL US Equity,93728300.0
AMGN US Equity,1875926.0
AMZN US Equity,35601907.0
AXP US Equity,2426689.0
BA US Equity,4789953.0


---
<a id="bql-for-equity-univ-peers" />

### Building a universe from peers of an equity ticker

The [univ.peers()](https://help.bquant.blpprofessional.com/bql/universe_functions/peers) accessor can be used to return the peers of a chosen equity ticker.

Use the `type` parameter to specify the source from which to pull peers:

- Passing `'CUSTOM'` (default) returns the last custom peer group you selected on a relative analysis function on the Bloomberg® Terminal, such as as [RV&lt;GO&gt;](https://blinks.bloomberg.com/screens/rv) or [EQRV&lt;GO&gt;](https://blinks.bloomberg.com/screens/eqrv).
- Passing `'BLOOMBERG_BEST_FIT'` returns a proprietary peer group curated by a Bloomberg Intelligence (BI) analyst or created with a Bloomberg algorithm (BICS Best Fit).

This example gets the peers of Vodafone as defined by the Bloomberg Best Fit algorith and queries BQL for the PE ratio and adjusted EPS. Using `currency='USD'` returns the values for the data item in USD.

In [20]:
universe = bq.univ.peers('VOD LN Equity', type='BLOOMBERG_BEST_FIT')
data_items = {
    'P/E': bq.data.pe_ratio(),
    'EPS': bq.data.is_eps(currency='USD', fa_adjusted='Y'),
}

request = bql.Request(universe, data_items)
response = bq.execute(request)

data = pd.concat(
    [data_item.df()[data_item.name] for data_item in response],
    axis=1
)
data

,P/E,EPS
ID,,
VOD LN Equity,16.610776,0.059420
RTKM RM Equity,NaN,0.078808
VEON US Equity,7.704140,3.764210
TEL2B SS Equity,20.411884,0.552400
TKA AV Equity,8.411676,1.130261
TCELL TI Equity,35.489366,0.103225
MTSS RM Equity,NaN,0.440918
TTKOM TI Equity,13.113161,0.146460
OPL PW Equity,13.681134,0.157413


---
<a id="bql-for-equity-univ-value" />

### Importing private portfolios from the Bloomberg Terminal®

You can use BQL to import your Terminal based portfolios -- including any portfolios you created with the Portfolio Administration function ([PRTU&lt;GO&gt;](https://blinks.bloomberg.com/screens/prtu)) or uploaded via the Bloomberg Uploader ([BBU&lt;GO&gt;](https://blinks.bloomberg.com/screens/bbu)).

To import a private portfolio, you'll need the portfolio's ID, which you can find in the upper-right corner of the Terminal window when viewing the portfolio in [PRTU&lt;GO&gt;](https://blinks.bloomberg.com/screens/prtu).

In this two-part example, the `type='PORT'` argument imports private portfolio. Replace `'your_portfolio_id'` with a string containing your portfolio's ID.

In [21]:
portfolio = bq.univ.members('U14000284-200', type='PORT')
data_items = {
    '1M Return': bq.data.total_return(calc_interval='1M'),
    'Weights': bq.data.id()['WEIGHTS'],   
    'Mkt Cap' : bq.data.cur_mkt_cap(currency='USD'),
    'P/E': bq.data.pe_ratio(),
    'EPS': bq.data.is_eps(currency='USD', fa_adjusted='Y'),
}

To stop this notebook from experiencing an error, the second half of the example has been commented out. Uncomment the following code cell after replacing `'your_portfolio_id'` in the above code cell.

In [22]:
# request = bql.Request(portfolio, data_items)
# response = bq.execute(request)

# data = pd.concat(
#     [field.df()[field.name] for field in response],
#     axis=1
# )
# data.head()

In [23]:
request = bql.Request(portfolio, data_items)
response = bq.execute(request)

data = pd.concat(
     [field.df()[field.name] for field in response],
     axis=1
 )
data.head()

,1M Return,Weights,Mkt Cap,P/E,EPS
ID,,,,,
AAPL UW Equity,0.015538,3.652347,3.358746e+12,33.624049,6.590000
AMGN UW Equity,0.025629,5.388497,1.751263e+11,37.119790,8.830933
AXP UN Equity,0.058511,4.148508,1.783820e+11,18.725373,13.430000
BA UN Equity,-0.007433,2.693422,1.003798e+11,NaN,-7.676614
CAT UN Equity,-0.005922,5.522746,1.619754e+11,14.704570,22.831467


---
<a id="bql-for-equity-univ-value" />

## Filtering within Pandas Dataframe

You can use logical operators ( > < == !=) on the column values to filter for rows

In [24]:
# Filter the dataframe for companies with Market Cap > 10B USD 

data[(data['Mkt Cap'] > 10^9)]

,1M Return,Weights,Mkt Cap,P/E,EPS
ID,,,,,
AAPL UW Equity,0.015538,3.652347,3.358746e+12,33.624049,6.590000
AMGN UW Equity,0.025629,5.388497,1.751263e+11,37.119790,8.830933
AXP UN Equity,0.058511,4.148508,1.783820e+11,18.725373,13.430000
BA UN Equity,-0.007433,2.693422,1.003798e+11,NaN,-7.676614
CAT UN Equity,-0.005922,5.522746,1.619754e+11,14.704570,22.831467
CRM UN Equity,-0.022084,4.063196,2.349466e+11,40.487291,6.136205
CSCO UW Equity,0.085600,0.805166,1.943488e+11,16.999875,2.878813
CVX UN Equity,-0.021807,2.319606,2.565971e+11,12.726026,11.048566
DIS UN Equity,0.027807,1.460542,1.602123e+11,22.450060,3.948111


In [25]:
# Filter the dataframe for companies with Market Cap > 10B USD AND PE ratio > 30

data[(data['Mkt Cap'] > 10^9) & (data['P/E'] > 30 )]

,1M Return,Weights,Mkt Cap,P/E,EPS
ID,,,,,
AAPL UW Equity,0.015538,3.652347,3.358746e+12,33.624049,6.590000
AMGN UW Equity,0.025629,5.388497,1.751263e+11,37.119790,8.830933
CRM UN Equity,-0.022084,4.063196,2.349466e+11,40.487291,6.136205
INTC UW Equity,-0.014979,0.315288,8.154332e+10,37.329063,0.519021
MSFT UW Equity,-0.000903,6.707845,3.015732e+12,34.346722,11.873129
V UN Equity,0.098965,4.722044,5.788352e+11,30.321780,12.023347
WMT UN Equity,0.128945,1.278677,6.216783e+11,32.506675,2.386321


In [26]:
# Filter the dataframe for companies with Market Cap > 10B USD OR PE ratio > 30

data[(data['Mkt Cap'] > 10^9) | (data['P/E'] > 30 )]

,1M Return,Weights,Mkt Cap,P/E,EPS
ID,,,,,
AAPL UW Equity,0.015538,3.652347,3.358746e+12,33.624049,6.590000
AMGN UW Equity,0.025629,5.388497,1.751263e+11,37.119790,8.830933
AXP UN Equity,0.058511,4.148508,1.783820e+11,18.725373,13.430000
BA UN Equity,-0.007433,2.693422,1.003798e+11,NaN,-7.676614
CAT UN Equity,-0.005922,5.522746,1.619754e+11,14.704570,22.831467
CRM UN Equity,-0.022084,4.063196,2.349466e+11,40.487291,6.136205
CSCO UW Equity,0.085600,0.805166,1.943488e+11,16.999875,2.878813
CVX UN Equity,-0.021807,2.319606,2.565971e+11,12.726026,11.048566
DIS UN Equity,0.027807,1.460542,1.602123e+11,22.450060,3.948111


In [27]:
# Filter the dataframe for companies with Market Cap > 10B USD OR PE ratio > 30 BUT must have positive 1M Return

data[(data['1M Return'] > 0) & ((data['Mkt Cap'] > 10^9) | (data['P/E'] > 30 )) ]

,1M Return,Weights,Mkt Cap,P/E,EPS
ID,,,,,
AAPL UW Equity,0.015538,3.652347,3.358746e+12,33.624049,6.590000
AMGN UW Equity,0.025629,5.388497,1.751263e+11,37.119790,8.830933
AXP UN Equity,0.058511,4.148508,1.783820e+11,18.725373,13.430000
CSCO UW Equity,0.085600,0.805166,1.943488e+11,16.999875,2.878813
DIS UN Equity,0.027807,1.460542,1.602123e+11,22.450060,3.948111
GS UN Equity,0.012240,8.077620,1.599341e+11,15.507926,31.958412
HD UN Equity,0.063476,6.043211,3.630686e+11,24.597577,14.910000
HON UW Equity,0.027768,3.325816,1.306879e+11,20.347549,9.950616
IBM UN Equity,0.074150,3.365000,1.874813e+11,21.749219,9.487111


We will be covering more filtering workflows in BQL in session 2 of our BQNT Equity series.

---
<a id="bql-for-equity-aggregating" />

## Aggregating and ranking data

### Performing aggregate analyses with `group()`

The [func.group()](https://help.bquant.blpprofessional.com/bql/functions/group) method lets you use BQL to partition your target universe into subgroups. You can then chain a [BQL function](https://help.bquant.blpprofessional.com/content?id=QMTY6B2aNygh3N9CA6Laek) to perform aggregate operations over each subgroup. 

This example calculates the average annual return by sector for members of the Dow Jones Industrial Average.

In [28]:
universe = bq.univ.members('INDU Index')

# Get the 1-year return for every ticker in the universe
return_1y = bq.data.total_return(calc_interval='1Y')
# Get the Bloomberg Industry Classification System for later grouping 
sector = bq.data.classification_name(classification_scheme='BICS')
# Calculate the average return by sector group
data_item = {'Avg Return': return_1y.group(sector).avg()}

request = bql.Request(universe, data_item)
response = bq.execute(request)

data = response[0].df()
data.head()

,ORIG_IDS,CLASSIFICATION_NAME(CLASSIFICATION_SCHEME='BICS'),Avg Return
ID,,,
Communications,None,Communications,0.214901
Consumer Discretionary,None,Consumer Discretionary,0.066991
Consumer Staples,None,Consumer Staples,0.285514
Energy,CVX UN Equity,Energy,-0.106330
Financials,None,Financials,0.475851


In the above code example, the `data_item` is created by doing the following:
1. Get the 1-year return for every ticker in the universe.
1. Group the tickers by sector.
1. Calculate the average return by sector group.

---
<a id="bql-for-equity-aggregating-wavg" />

### Calculating weighted average

The following example shows a deeper analysis making use of [func.group()](https://help.bquant.blpprofessional.com/bql/functions/group) with [func.wavg()](https://help.bquant.blpprofessional.com/bql/functions/wavg) to calculate the weighted average annual return by sector for tickers in the Dow Jones Industrial Average, weighted by their index weight.

In [29]:
universe = bq.univ.members('INDU Index')

# Get the 1-year return for every ticker in the universe
return_1y = bq.data.total_return(calc_interval='1Y')
# Get the Bloomberg Industry Classification System for later grouping 
sector = bq.data.classification_name(classification_scheme='BICS')
# Get the index weights and group by sector
grouped_wgts = bq.data.id()['WEIGHTS'].group(sector)
# Calculate the average return by sector group, using identified index weights
data_item = {
    'Wgtd Avg Return': return_1y.group(by=sector).wavg(weights=grouped_wgts)
}

request = bql.Request(universe, data_item)
response = bq.execute(request)

data = response[0].df()
data.head()

,ORIG_IDS,CLASSIFICATION_NAME(CLASSIFICATION_SCHEME='BICS'),Wgtd Avg Return
ID,,,
Communications,None,Communications,0.167593
Consumer Discretionary,None,Consumer Discretionary,0.107057
Consumer Staples,None,Consumer Staples,0.249841
Energy,CVX UN Equity,Energy,-0.106330
Financials,None,Financials,0.478087


In the above code example, the `data_item` is created by doing the following:

1. Get the 1-year return for every ticker in the universe.
1. Group the tickers by sector.
1. Calculate the average return by sector group, weighted by the security weights in the index.

---
<a id="bql-for-equity-aggregating-cut" />

### Creating quantiles with `cut()`

Use [func.cut()](https://help.bquant.blpprofessional.com/bql/functions/cut) to bucket data items into a given number of bins containing the same number of data points (i.e., quantiles).

The parameter you pass to `cut()` is the number of bins you want to break your data set into, e.g., `cut(100)` creates percentiles and `cut(10)` creates deciles. Each bin created by `cut()` has a bin number to identify it, the lowest being `1`.

[func.ungroup()](https://help.bquant.blpprofessional.com/bql/functions/ungroup) ungroups the securities from their buckets. After running `func.ungroup()`, returned data is at a single-security level, not the aggregated bucket level.

The following example returns the EV to Sales ratio for the members of the Dow Jones Industrial Index, along with the percentile.

In [30]:
universe = bq.univ.members('INDU Index')

ev_to_sales = bq.data.ev_to_sales()
data_items = {
    'EV to Sales': ev_to_sales,
    'EV to Sales Quantile': ev_to_sales.group().cut(100).ungroup()
}

request = bql.Request(universe, data_items)
response  = bq.execute(request)

data = pd.concat(
    [data_item.df()[data_item.name] for data_item in response],
    axis=1
)
data.sort_values('EV to Sales Quantile', ascending=False).head()

,EV to Sales,EV to Sales Quantile
ID,,
V UN Equity,16.503930,100.0
MSFT UQ Equity,12.393997,97.0
MCD UN Equity,10.100473,93.0
AAPL UQ Equity,8.576201,90.0
AMGN UQ Equity,7.385735,86.0


The data item definition `ev_to_sales.group().cut(100).ungroup()` does the following:

1. Define the data returned from `bq.data.ev_to_sales()` as a single group using [`func.group()`](https://help.bquant.blpprofessional.com/bql/functions/group).
1. Calculate the quantile (percentile) of each security in the group using [`func.cut(100)`](https://help.bquant.blpprofessional.com/bql/functions/cut). Note: If multiple securities have the same quantile, they're in the same bucket.
1. Ungroup the securities from their buckets using [`func.ungroup()`](https://help.bquant.blpprofessional.com/bql/functions/ungroup).

---
<a id="bql-for-equity-aggregating-group-family" />

### Calculating $z$-scores

A $z$-score is a statistical measure of a value's distance from the mean of the data set it belongs to. You can use [func.groupzscore()](https://help.bquant.blpprofessional.com/bql/functions/groupzscore) to calculate the $z$-scores for each value in a grouped data set.

In this example, we use the EPS then `groupzscore()` by sector to see how many standard deviations from the mean each security's value is relative to others in its sector.

In [31]:
universe = bq.univ.members('INDU Index')

sector = bq.data.classification_name(classification_scheme='BICS')
# Calculate the z-score of EPS by sector
data_items = {
    'Sector': sector,
    'EPS': bq.data.is_eps(),
    'EPS z-score': bq.data.is_eps().groupzscore(by=sector)
}

request = bql.Request(universe, data_items)
response = bq.execute(request)

data = pd.concat(
    [data_item.df()[data_item.name] for data_item in response],
    axis=1
)
data.head()

,Sector,EPS,EPS z-score
ID,,,
AXP UN Equity,Financials,13.43,-0.594533
VZ UN Equity,Communications,2.67,0.707107
BA UN Equity,Industrials,-5.63,-1.048385
CAT UN Equity,Industrials,22.07,1.299080
JPM UN Equity,Financials,17.95,-0.021677


---
<a id="bql-for-equity-aggregating-grouprank" />

### Using `grouprank()`

The [func.grouprank()](https://help.bquant.blpprofessional.com/bql/functions/grouprank) method sorts universe members based on some attribute and returns a ranked set of rows.

This example displays each security in the Dow Jones Industrial Average's rank of three years earnings growth against other companies in its sector.

In [32]:
universe = bq.univ.members('INDU Index')

# Precent change of annual EPS over last 3 years
eps_3y_growth = bq.data.is_eps(
    fa_period_offset=bq.func.range(-2, 0),
    fa_period_type='A'
).pct_chg()

sector = bq.data.classification_name()
data_items = {
    'Sector': sector,
    'Earnings Growth 3Y': eps_3y_growth,
    'Earnings Growth 3Y Sector Rank': eps_3y_growth.grouprank(by=sector)
}

request = bql.Request(universe, data_items)
response = bq.execute(request)

data = pd.concat(
    [data_item.df()[data_item.name] for data_item in response],
    axis=1
)
data.head()

,Sector,Earnings Growth 3Y,Earnings Growth 3Y Sector Rank
ID,,,
AXP UN Equity,Financials,11.852590,2.0
VZ UN Equity,Communications,-48.120301,2.0
BA UN Equity,Industrials,-48.671329,3.0
CAT UN Equity,Industrials,69.656329,1.0
JPM UN Equity,Financials,5.588044,3.0


---
<a id="bql-for-equity-univ-filtering" />

More functions can be found on BQLX under Function Reference.